In [1]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=b9c82521ad3120549144f5e24ad7070ac85e5c2c554553132554b56123f2f6f5
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install language_tool_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
# !pip install librosa nltk language-tool-python fasttext
import os
import numpy as np
import librosa
import nltk
import re
import fasttext
import fasttext.util
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from langdetect import detect
import language_tool_python
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Concatenate, Flatten

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os

2025-05-12 11:24:45.016167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747049085.336596      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747049085.413009      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import os
import numpy as np
import librosa
import pandas as pd

# 1. Load the audio file
def load_patient_audio(audio_path, timestamps):
    # Read the audio file
    audio, sr = librosa.load(audio_path, sr=16000)
    
    # Extract patient speech segments based on timestamps
    patient_audio = []
    for start_time, end_time in timestamps:
        start_sample = int(start_time * sr)
        end_sample = int(end_time * sr)
        patient_audio.append(audio[start_sample:end_sample])
    
    # Concatenate all patient audio segments into one array
    patient_audio = np.concatenate(patient_audio)
    return patient_audio, sr

# 2. Get the patient timestamps from the transcript file
def get_patient_timestamps(transcript_path):
    # Read the CSV file
    df = pd.read_csv(transcript_path, sep="\t")  # Use tab-separated values
    df.columns = df.columns.str.lower()  # Normalize column names to lowercase
    
    # Filter rows for 'participant' speech
    patient_segments = df[df['speaker'].str.lower() == 'participant'][['start_time', 'stop_time']]
    
    return list(patient_segments.itertuples(index=False, name=None))

# 3. Divide audio into 7.6s segments
def segment_audio(audio, sr, segment_duration=7.6):
    segment_samples = int(segment_duration * sr)
    segments = [audio[i:i+segment_samples] for i in range(0, len(audio), segment_samples)]
    
    # Ensure that the last segment is not too short
    if len(segments[-1]) < segment_samples:
        segments = segments[:-1]
        
    return segments

# 4. Extract MFCC features with adjusted hop length
def extract_mfcc(segments, sr, n_mfcc=60, n_fft=1024, hop_length=322, win_length=960):
    mfcc_features = []
    
    for segment in segments:
        # Extract MFCCs with the defined parameters
        mfcc = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
        mfcc_features.append(mfcc.T)  # Transpose to match (time, n_mfcc)
    
    # Stack all segments' MFCCs
    mfcc_features = np.array(mfcc_features)
    
    return mfcc_features

# 5. Apply Data Augmentation (optional)
def augment_audio(segments, sr, noise_factor=0.005):
    augmented_segments = []
    
    for segment in segments:
        # Add random noise to the audio segment
        noise = np.random.randn(len(segment))
        augmented_segment = segment + noise_factor * noise
        augmented_segments.append(augmented_segment)
    
    return augmented_segments

# Path to the audio and transcript
audio_path = "/kaggle/input/daic-woz/354_P/354_AUDIO.wav"
transcript_path = "/kaggle/input/daic-woz/354_P/354_TRANSCRIPT.csv"

# 6. Process the audio
timestamps = get_patient_timestamps(transcript_path)
patient_audio, sr = load_patient_audio(audio_path, timestamps)

# 7. Segment the audio into 7.6s segments
segments = segment_audio(patient_audio, sr)

# 8. Augment the audio (optional)
augmented_segments = augment_audio(segments, sr)

# 9. Extract MFCC features
mfcc_features = extract_mfcc(augmented_segments, sr)

In [5]:
# 10. Check results
print(f"Sample rate: {sr}")
print(f"Total patient audio duration (s): {len(patient_audio) / sr:.2f}")
print(f"Number of 7.6s segments: {len(mfcc_features)}")
print(f"Shape of MFCC for one segment: {mfcc_features[0].shape}")
print(f"All MFCC features shape: {mfcc_features.shape}")

Sample rate: 16000
Total patient audio duration (s): 146.38
Number of 7.6s segments: 19
Shape of MFCC for one segment: (378, 60)
All MFCC features shape: (19, 378, 60)


# **TEXT******

In [6]:
import pandas as pd
import numpy as np

transcript_df = pd.read_csv('/kaggle/input/daic-woz/354_P/354_TRANSCRIPT.csv', sep='\t')
print(transcript_df.columns)
transcript_df = transcript_df.dropna(subset=['value'])  # remove empty rows

Index(['start_time', 'stop_time', 'speaker', 'value'], dtype='object')


In [7]:
segment_duration = 7.6
max_words_per_segment = 9

# Get total time
total_time = transcript_df['stop_time'].max()
n_segments = int(np.ceil(total_time / segment_duration))

# Create list of transcripts per segment
segmented_transcripts = []

for i in range(n_segments):
    start = i * segment_duration
    end = (i + 1) * segment_duration
    segment = transcript_df[(transcript_df['start_time'] >= start) & (transcript_df['stop_time'] < end)]
    # Keep only participant + interviewer (optional)
    segment_text = " ".join(segment['value'].astype(str).tolist())
    segmented_transcripts.append(segment_text)

In [8]:
import numpy as np

# Load GloVe model
def load_glove_model(file_path):
    glove_model = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            split_line = line.strip().split()
            word = split_line[0]
            vector = np.asarray(split_line[1:], dtype='float32')
            glove_model[word] = vector
    return glove_model

# Load the GloVe embeddings (replace 'path_to_glove_file' with the actual file path)
glove_model = load_glove_model('/kaggle/input/glove-model/glove.6B.100d.txt')

In [9]:
import numpy as np

def text_to_embedding(text, model, max_words=9):
    words = text.split()
    vecs = [glove_model.get(w.lower(), np.zeros(100)) for w in words[:max_words]]
    
    # Pad with zero vectors if there are fewer than `max_words` words
    while len(vecs) < max_words:
        vecs.append(np.zeros(100))
    
    # Stack into (9, 100), then transpose to (100, 9)
    emb = np.stack(vecs, axis=0).T  # (100, 9)
    
    # Pad to (378, 9)
    if emb.shape[0] < 378:
        pad_rows = np.zeros((378 - emb.shape[0], emb.shape[1]))
        emb = np.vstack((emb, pad_rows))  # (378, 9)
    
    return emb

In [10]:
X_text_segments = np.array([text_to_embedding(text,glove_model) for text in segmented_transcripts])  # (n_segments, 378, 9)

In [11]:
# Convert to NumPy array: (num_segments, 9, 300)
X_text_segments = np.array(X_text_segments)

print(f"X_text_segments shape: {X_text_segments.shape}")

X_text_segments shape: (73, 378, 9)


In [12]:
np.save("354_text_features.npy", X_text_segments)

# **NEW**

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers
import numpy as np

# Simulated input shapes based on provided dimensions
# Audio: (378, 60), Text (Word2Vec): (378, 9)
input_shape_audio = (378, 60)
input_shape_text = (378, 9)

In [15]:
def build_mfcc_cnn_word2vec_early():
    # MFCC branch
    input_audio = layers.Input(shape=input_shape_audio)
    x_audio = layers.Reshape((378, 60, 1))(input_audio)
    x_audio = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x_audio)
    x_audio = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x_audio)
    x_audio = layers.Dropout(0.0002)(x_audio)
    x_audio = layers.Flatten()(x_audio)

    # Text branch
    input_text = layers.Input(shape=input_shape_text)
    x_text = layers.Flatten()(input_text)

    # Concatenate
    x = layers.Concatenate()([x_audio, x_text])
    x = layers.Dense(15, activation='tanh')(x)
    x = layers.Dense(10, activation='tanh')(x)
    output = layers.Dense(2, activation='sigmoid')(x)

    model = models.Model(inputs=[input_audio, input_text], outputs=output)
    return model

In [16]:
def build_mfcc_lstm_word2vec_early():
    input_audio = layers.Input(shape=input_shape_audio)
    x_audio = layers.LSTM(60, activation='tanh', recurrent_activation='hard_sigmoid',
                          recurrent_dropout=0.002, kernel_initializer='glorot_uniform', return_sequences=True)(input_audio)
    x_audio = layers.BatchNormalization()(x_audio)
    x_audio = layers.Dropout(0.0002)(x_audio)

    x_audio = layers.LSTM(40, activation='tanh', recurrent_activation='hard_sigmoid',
                          recurrent_dropout=0.002, kernel_initializer='glorot_uniform', return_sequences=True)(x_audio)
    x_audio = layers.BatchNormalization()(x_audio)
    x_audio = layers.Dropout(0.0002)(x_audio)

    x_audio = layers.LSTM(20, activation='tanh', recurrent_activation='sigmoid',
                          recurrent_dropout=0.002, kernel_initializer='glorot_uniform')(x_audio)
    x_audio = layers.BatchNormalization()(x_audio)
    x_audio = layers.Dropout(0.0002)(x_audio)

    input_text = layers.Input(shape=input_shape_text)
    x_text = layers.Flatten()(input_text)

    x = layers.Concatenate()([x_audio, x_text])
    x = layers.Dense(15, activation='tanh')(x)
    x = layers.Dense(10, activation='tanh')(x)
    output = layers.Dense(2, activation='sigmoid')(x)

    model = models.Model(inputs=[input_audio, input_text], outputs=output)
    return model

In [17]:
def build_mfcc_cnn_word2vec_model_level():
    input_audio = layers.Input(shape=input_shape_audio)
    x_audio = layers.Reshape((378, 60, 1))(input_audio)
    x_audio = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x_audio)
    x_audio = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x_audio)
    x_audio = layers.Dropout(0.0002)(x_audio)
    x_audio = layers.Flatten()(x_audio)
    x_audio = layers.Dense(20, activation='relu')(x_audio)

    input_text = layers.Input(shape=input_shape_text)
    x_text = layers.Flatten()(input_text)
    x_text = layers.Dense(20, activation='relu')(x_text)

    x = layers.Concatenate()([x_audio, x_text])
    x = layers.Dense(10, activation='tanh')(x)
    output = layers.Dense(2, activation='sigmoid')(x)

    model = models.Model(inputs=[input_audio, input_text], outputs=output)
    return model

In [18]:
def build_mfcc_lstm_word2vec_model_level():
    input_audio = layers.Input(shape=input_shape_audio)
    x_audio = layers.LSTM(60, activation='tanh', recurrent_activation='hard_sigmoid',
                          recurrent_dropout=0.002, kernel_initializer='glorot_uniform', return_sequences=True)(input_audio)
    x_audio = layers.BatchNormalization()(x_audio)
    x_audio = layers.Dropout(0.0002)(x_audio)

    x_audio = layers.LSTM(40, activation='tanh', recurrent_activation='hard_sigmoid',
                          recurrent_dropout=0.002, kernel_initializer='glorot_uniform', return_sequences=True)(x_audio)
    x_audio = layers.BatchNormalization()(x_audio)
    x_audio = layers.Dropout(0.0002)(x_audio)

    x_audio = layers.LSTM(20, activation='tanh', recurrent_activation='sigmoid',
                          recurrent_dropout=0.002, kernel_initializer='glorot_uniform')(x_audio)
    x_audio = layers.BatchNormalization()(x_audio)
    x_audio = layers.Dropout(0.0002)(x_audio)

    input_text = layers.Input(shape=input_shape_text)
    x_text = layers.Flatten()(input_text)

    x = layers.Concatenate()([x_audio, x_text])
    x = layers.Reshape((1, -1))(x)
    x = layers.LSTM(20, activation='tanh', return_sequences=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.0002)(x)
    x = layers.Flatten()(x)

    x = layers.Dense(15, activation='tanh')(x)
    x = layers.Dense(10, activation='tanh')(x)
    output = layers.Dense(2, activation='sigmoid')(x)

    model = models.Model(inputs=[input_audio, input_text], outputs=output)
    return model

In [19]:
# Create all models
models_dict = {
    "MFCC-CNN Word2Vec (Early Fusion)": build_mfcc_cnn_word2vec_early(),
    "MFCC-LSTM Word2Vec (Early Fusion)": build_mfcc_lstm_word2vec_early(),
    "MFCC-CNN Word2Vec (Model-Level Fusion)": build_mfcc_cnn_word2vec_model_level(),
    "MFCC-LSTM Word2Vec (Model-Level Fusion)": build_mfcc_lstm_word2vec_model_level()
}

I0000 00:00:1747049128.783339      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747049128.784115      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [20]:
# Compile them with RMSProp and RMSE loss (approximated with MSE here for demo)
for name, model in models_dict.items():
    model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-6, decay=1e-6),
                  loss='mse',
                  metrics=['mae'])

models_dict.keys() 

/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


dict_keys(['MFCC-CNN Word2Vec (Early Fusion)', 'MFCC-LSTM Word2Vec (Early Fusion)', 'MFCC-CNN Word2Vec (Model-Level Fusion)', 'MFCC-LSTM Word2Vec (Model-Level Fusion)'])

In [21]:
# Assuming you're using Keras/TensorFlow models
models_dict = {
    "MFCC-CNN Word2Vec (Early Fusion)": build_mfcc_cnn_word2vec_early(),
    "MFCC-LSTM Word2Vec (Early Fusion)": build_mfcc_lstm_word2vec_early(),
    "MFCC-CNN Word2Vec (Model-Level Fusion)": build_mfcc_cnn_word2vec_model_level(),
    "MFCC-LSTM Word2Vec (Model-Level Fusion)": build_mfcc_lstm_word2vec_model_level()
}

# Print summary of each model
for model_name, model in models_dict.items():
    print(f"Summary of {model_name}:")
    model.summary()
    print("\n" + "-"*80 + "\n")

Summary of MFCC-CNN Word2Vec (Early Fusion):


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 378, 60)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_3 (Reshape)       │ (None, 378, 60, 1)     │              0 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 378, 60, 128)   │          1,280 │ reshape_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 378, 60, 64)    │         73,792 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_9 (Dropout)       │ (None, 378, 60, 64)    │              0 │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_9             │ (None, 378, 9)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_7 (Flatten)       │ (None, 1451520)        │              0 │ dropout_9[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_8 (Flatten)       │ (None, 3402)           │              0 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, 1454922)        │              0 │ flatten_7[0][0],       │
│ (Concatenate)             │                        │                │ flatten_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 15)             │     21,823,845 │ concatenate_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_14 (Dense)          │ (None, 10)             │            160 │ dense_13[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_15 (Dense)          │ (None, 2)              │             22 │ dense_14[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 21,899,099 (83.54 MB)

 Trainable params: 21,899,099 (83.54 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14            │ (None, 378, 60)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_10 (LSTM)            │ (None, 378, 60)        │         29,040 │ input_layer_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_10    │ (None, 378, 60)        │            240 │ lstm_10[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_14 (Dropout)      │ (None, 378, 60)        │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_11 (LSTM)            │ (None, 378, 40)        │         16,160 │ dropout_14[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_11    │ (None, 378, 40)        │            160 │ lstm_11[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_15 (Dropout)      │ (None, 378, 40)        │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_12 (LSTM)            │ (None, 20)             │          4,880 │ dropout_15[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_12    │ (None, 20)             │             80 │ lstm_12[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_15            │ (None, 378, 9)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_16 (Dropout)      │ (None, 20)             │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_12 (Flatten)      │ (None, 3402)           │              0 │ input_layer_15[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_7             │ (None, 3422)           │              0 │ dropout_16[0][0],      │
│ (Concatenate)             │                        │                │ flatten_12[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_5 (Reshape)       │ (None, 1, 3422)        │              0 │ concatenate_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_13 (LSTM)            │ (None, 20)             │        275,440 │ reshape_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_13    │ (None, 20)             │             80 │ lstm_13[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 326,577 (1.25 MB)

 Trainable params: 326,297 (1.24 MB)

 Non-trainable params: 280 (1.09 KB)


--------------------------------------------------------------------------------



# Training and evaluation

In [22]:
!pip install tensorflow scikit-learn torch torchsummary
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-10.3.9.90:
      Successfully uninstalled nvidia-curand-cu12-10.3.9.90
  Attemptin

In [23]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error
from scipy.stats import pearsonr

In [24]:
def ccc(true, pred):
    """Concordance Correlation Coefficient"""
    mean_true = np.mean(true)
    mean_pred = np.mean(pred)
    var_true = np.var(true)
    var_pred = np.var(pred)
    std_true = np.std(true)
    std_pred = np.std(pred)
    corr, _ = pearsonr(true, pred)

    numerator = 2 * corr * std_true * std_pred
    denominator = (std_true**2 + std_pred**2) + (mean_true - mean_pred)**2
    return numerator / denominator

In [25]:
# Helper function to train and evaluate the models
def train_and_evaluate_model(model, train_data, val_data, batch_size=120, epochs=300, learning_rate=1e-5):
    # Assuming train_data and val_data are tuples (X_train, y_train) and (X_val, y_val)
    X_train, y_train = train_data
    X_val, y_val = val_data
    
    # Compile model with RMSProp optimizer and RMSE loss
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate), 
                  loss='mean_squared_error', 
                  metrics=['accuracy'])
    
    # Train model with early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                        validation_data=(X_val, y_val), callbacks=[early_stopping])
    
    # Evaluate the model on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate the evaluation metrics
    accuracy = accuracy_score(y_val, np.round(y_pred))
    precision = precision_score(y_val, np.round(y_pred), average='binary')
    recall = recall_score(y_val, np.round(y_pred), average='binary')
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    pearson_corr, _ = pearsonr(y_val, y_pred)
    ccc_value = ccc(y_val, y_pred)

    return accuracy, precision, recall, rmse, pearson_corr, ccc_value

In [26]:
# Assuming you have your training and validation data (X_train, y_train, X_val, y_val)
# Example: train_data = (X_train, y_train), val_data = (X_val, y_val)

# Loop through each model and evaluate
for model_name, model in models_dict.items():
    print(f"Training and evaluating {model_name}...")
    
    # Define train and validation data here
    # Example (replace these with actual data):
    train_data = (X_train, y_train)  # Replace with your actual training data
    val_data = (X_val, y_val)        # Replace with your actual validation data
    
    accuracy, precision, recall, rmse, pearson_corr, ccc_value = train_and_evaluate_model(model, train_data, val_data)
    
    print(f"Results for {model_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"Pearson Correlation Coefficient: {pearson_corr:.4f}")
    print(f"Concordance Correlation Coefficient (CCC): {ccc_value:.4f}")
    print("-" * 80)

Training and evaluating MFCC-CNN Word2Vec (Early Fusion)...


NameError: name 'X_train' is not defined